In [1]:
import datetime
import gc
import joblib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import scipy.stats
import seaborn as sns
import sys
sys.path.append("../../")
import time
import warnings
warnings.simplefilter("ignore")
from collections import defaultdict
from itertools import repeat, combinations
from pandarallel import pandarallel
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tqdm import tqdm

In [2]:
from utils.common import (
    sigmoid, pad_column_name
)
from utils.constants import *
from utils.eval_helpers import (
    plot_roc_curves, plot_feature_importance, 
    amex_metric, get_final_metric_df, amex_metric_np, lgb_amex_metric
)
from utils.eda_helpers import (
    plot_missing_proportion_barchart, plot_heatmap,
    get_cols, insert_row_number, plot_train_test_distribution
)
from utils.extraction_helpers import read_file
from utils.feature_group import CATEGORY_COLUMNS
from utils.preprocess_helpers import clip_col

In [3]:
from feature_engineering_helpers import feature_gen_pipeline, clip_all, round_all, convert_all

In [4]:
%load_ext autoreload
%autoreload

In [5]:
pandarallel.initialize(nb_workers=16, progress_bar=False, use_memory_fs=False)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [6]:
START = time.time()

### Read Data

#### Impute Train

In [7]:
labels = read_file(f"../{RAW_DATA_PATH}/train_labels.csv")

Shape of data: (458913, 2)


In [8]:
%%time
# train = read_file(f"../{INTERIM_DATA_PATH}/v5/train_parquet/train_all_variables.parquet")
train = read_file(f"./train_imputed.parquet")

Shape of data: (5531451, 189)
CPU times: user 16.7 s, sys: 29.2 s, total: 45.9 s
Wall time: 25 s


In [80]:
missing = train.isnull().sum()
missing_columns = missing[missing > 0].index.tolist()
len(missing_columns)

38

In [42]:
%%time
for col in tqdm(missing_columns):
    train[col] = train.groupby("customer_ID")[col].bfill().ffill()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 116/116 [01:35<00:00,  1.21it/s]

CPU times: user 1min 14s, sys: 18.9 s, total: 1min 32s
Wall time: 1min 35s


In [43]:
len(missing_columns)

116

In [86]:
# train.to_parquet(f"./train_imputed.parquet")

In [45]:
gc.collect()

8358

#### Impute Test

In [83]:
%%time
test1 = read_file(f"../{INTERIM_DATA_PATH}/v5/test_parquet/test_all_variables1.parquet")
test2 = read_file(f"../{INTERIM_DATA_PATH}/v5/test_parquet/test_all_variables2.parquet")
test = pd.concat([test1, test2], ignore_index=True)
del test1, test2

Shape of data: (5681079, 188)
Shape of data: (5682683, 188)
CPU times: user 12.2 s, sys: 22.9 s, total: 35.1 s
Wall time: 28.4 s


In [84]:
missing = test.isnull().sum()
missing_columns = missing[missing > 0].index.tolist()
len(missing_columns)

120

In [48]:
%%time
for col in tqdm(missing_columns):
    test[col] = test.groupby("customer_ID")[col].bfill().ffill()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [02:40<00:00,  1.34s/it]

CPU times: user 2min 30s, sys: 8.95 s, total: 2min 39s
Wall time: 2min 40s


In [85]:
# test.to_parquet(f"./test_imputed.parquet")

Shape of data: (11363762, 188)


In [51]:
# %%time
# n = train.nunique()
# binary_features = n[n == 2].index.tolist()

### Calculate diff for each column

In [10]:
# %%time
# train = read_file(f"./train_imputed.parquet")
# test = read_file(f"./test_imputed.parquet")

In [12]:
train = train.drop(columns=CATEGORY_COLUMNS, errors="ignore")

In [14]:
base_feature_column = train.iloc[:, 5:].columns.tolist()

In [15]:
for col in tqdm(base_feature_column):
    train[col] = train.groupby("customer_ID")[col].diff()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 173/173 [2:08:15<00:00, 44.48s/it]


In [16]:
train[col]

0          NaN
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
5531446    0.0
5531447    0.0
5531448    0.0
5531449    0.0
5531450    0.0
Name: D_145, Length: 5531451, dtype: float64

In [17]:
%%time
train.to_parquet("./train_diff.parquet")

CPU times: user 27.2 s, sys: 9.04 s, total: 36.3 s
Wall time: 26.7 s


In [18]:
test = read_file(f"./test_imputed.parquet")

Shape of data: (11363762, 188)


In [19]:
test = test.drop(columns=CATEGORY_COLUMNS, errors="ignore")

In [20]:
for col in tqdm(base_feature_column):
    test[col] = test.groupby("customer_ID")[col].diff()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 173/173 [4:20:15<00:00, 90.26s/it]


In [21]:
%%time
test.to_parquet("./test_diff.parquet")

CPU times: user 1min 3s, sys: 36.9 s, total: 1min 40s
Wall time: 1min 22s


In [29]:
train = train.loc[(train["row_number_inv"] != 1) | (train["row_number"] == 1)].reset_index(drop=True)

In [30]:
train.to_parquet("./train_diff.parquet")

In [24]:
test = test.loc[(test["row_number_inv"] != 1) | (test["row_number"] == 1)].reset_index(drop=True)

In [28]:
test.to_parquet("./test_diff.parquet")

### Transform Train Set

In [31]:
%%time
train_agg, keep_column = feature_gen_pipeline(train)

Done insertion
Average done
Minimum done
Maximum done
Standard Deviation done
Last entry done
First entry done
MA2 for Recency 1 done
MA2 for Recency 2 done
MA2 for Recency 3 done
MA3 for Recency 1 done
MA3 for Recency 2 done
MA3 for least Recency done


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 173/173 [01:00<00:00,  2.84it/s]

CPU times: user 1min 58s, sys: 1min 22s, total: 3min 21s
Wall time: 3min 48s


In [32]:
train_agg["target"] = labels["target"].values
print(f"Size: {sys.getsizeof(train_agg) / 1e9} GB, Shape: {train_agg.shape}")

Size: 10.344745409 GB, Shape: (458913, 3462)


In [33]:
train_agg.to_parquet("./train_agg_diff.parquet")

In [35]:
train_agg = convert_all(train_agg)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2767/2767 [02:14<00:00, 20.57it/s]


In [36]:
train_agg = clip_all(train_agg)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 107/107 [00:03<00:00, 30.12it/s]


In [37]:
train_agg = round_all(train_agg)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5019/5019 [00:39<00:00, 125.59it/s]


In [38]:
print(f"Size: {sys.getsizeof(train_agg) / 1e9} GB, Shape: {train_agg.shape}")

Size: 9.375529937 GB, Shape: (458913, 5063)


In [25]:
train_agg.shape

(458913, 5063)

In [40]:
# train_agg.to_parquet(f"./train_agg.parquet")

In [44]:
# del train_agg, train

### Transform Test

In [34]:
%%time
test_agg, keep_column = feature_gen_pipeline(test)

Done insertion
Average done
Minimum done
Maximum done
Standard Deviation done
Last entry done
First entry done
MA2 for Recency 1 done
MA2 for Recency 2 done
MA2 for Recency 3 done
MA3 for Recency 1 done
MA3 for Recency 2 done
MA3 for least Recency done


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 173/173 [01:24<00:00,  2.04it/s]

CPU times: user 3min 44s, sys: 4min 58s, total: 8min 42s
Wall time: 10min 53s


In [35]:
print(f"Size: {sys.getsizeof(test_agg) / 1e9} GB, Shape: {test_agg.shape}")

Size: 20.835015269 GB, Shape: (924621, 3461)


In [36]:
test_agg.to_parquet("./test_agg_diff.parquet")

In [ ]:
test_agg = convert_all(test_agg)

 28%|█████████████████████████████████▉                                                                                        | 772/2773 [03:01<07:30,  4.44it/s]

In [ ]:
test_agg = clip_all(test_agg)

In [ ]:
test_agg = round_all(test_agg)

In [ ]:
print(f"Size: {sys.getsizeof(test_agg) / 1e9} GB, Shape: {test_agg.shape}")

In [ ]:
test_agg.shape

In [40]:
# train_agg.to_parquet(f"./train_agg.parquet")

In [39]:
# del train_agg, train